In [1]:
from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/2.part'):
    shutil.rmtree('./file/2.part')

os.makedirs('./file/2.part')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

from selenium.webdriver import ChromeOptions

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import re

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/1.url'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

for file in list_file:
    
    # = = = = = = = = = = = = = = =

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./file/1.url/{file}',
                           header=0,
                           dtype=str).fillna('')

    input_['No'] = input_['No'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =

    option = ChromeOptions()
    option.add_experimental_option('useAutomationExtension', False)
    option.add_experimental_option('excludeSwitches', ['enable-automation'])
    option.add_argument('--disable-blink-features=AutomationControlled')
    option.add_argument('--disable-dev-shm-usage')
    option.add_argument('--disable-gpu')
    option.add_argument('--no-sandbox')
    # option.add_argument('--headless=new')
    # option.add_argument('blink-settings=imagesEnabled=false')
    option.add_argument(get_header(method='selenium'))
    option.add_argument(get_proxy(method='selenium'))
    option.page_load_strategy = 'eager'
    
    browser = Chrome(service=Service('../chromedriver'), options=option)
    browser.maximize_window()

    # = = = = = = = = = = = = = = =

    b = 0
    while True:
        b += 1

        try:
            browser.get('https://www.amazon.com/?language=en_US')
        
            break
        except KeyboardInterrupt:
            browser.quit()
            break
        except:
            browser.quit()
            continue

    # = = = = = = = = = = = = = = =

    input(f'''fuck-{file.removesuffix('.xlsx')}?: ''')
    
    # = = = = = = = = = = = = = = =

    for a in range(len(input_)):

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        try:
            b = 0
            while True:
                b += 1

                try:
                    request_url = input_.loc[a, 'Url']
                
                    browser.get(request_url)

                    soup = BeautifulSoup(browser.page_source, 'lxml')
                    html = etree.HTML(str(soup))

                    break
                except KeyboardInterrupt:
                    break
                except:
                    continue
        
            # = = = = = = = = = = = = = = =

            zip_code = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', html.xpath('//span[@id="glow-ingress-line2"]/text()')[0]).strip()
        
            # = = = = = = = = = = = = = = =

            price = html.xpath('//span[@class="a-offscreen"]/text()')[0].strip() if html.xpath('//span[@class="a-offscreen"]/text()') else ''

            # = = = = = = = = = = = = = = =

            dict_src, list_src = {}, json.loads('{"initial":[{' + browser.page_source.split('\'colorImages\': { \'initial\': [{')[1].split('}]},')[0] + '}]}')['initial']
            for i, src in enumerate(list_src):
                dict_src[str(i)] = src['hiRes'].strip() if src['hiRes'] else src['large'].strip()

            # = = = = = = = = = = = = = = =

            list_tr, dict_count = html.xpath('//table[@class="a-normal a-spacing-micro"]/tbody/tr'), {}
            for tr in list_tr:
                key = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./td/span/text()')[0]).strip()
                if key in dict_count:
                    dict_count[key] += 1
                else:
                    dict_count[key] = 1
            
            dict_overview, dict_no = {}, {}
            for i, tr in enumerate(list_tr):
                key = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./td/span/text()')[0]).strip()
                if dict_count[key] == 1:
                    dict_overview[str(i)] = {key: re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./td/span/text()')[1]).strip()}
                else:
                    if key in dict_no:
                        dict_no[key] += 1
                    else:
                        dict_no[key] = 1
                    dict_overview[str(i)] = {f'{key}-{dict_no[key]}': re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./td/span/text()')[1]).strip()}

            # = = = = = = = = = = = = = = =

            about, list_span = '', [re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', text).strip() for text in html.xpath('//ul[@class="a-unordered-list a-vertical a-spacing-mini"]/li/span/text()')]
            for i, span in zip(range(len(list_span)), list_span):
                about += f'{i+1}. {span}\n'
    
            # = = = = = = = = = = = = = = =

            dict_ = json.loads(re.sub(r'\\(?!["\\])', r'\\\\', browser.page_source.split('jQuery.parseJSON(\'')[1].split('}\');')[0] + '}'))

            # = = = = = = = = = = = = = = =

            title = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', dict_['title']).strip()

            # = = = = = = = = = = = = = = =

            parent_asin = dict_['parentAsin'].strip()

            # = = = = = = = = = = = = = = =
            
            list_tr, dict_count = html.xpath('//table[@id="productDetails_techSpec_section_1"]/tbody/tr'), {}
            for tr in list_tr:
                key = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./th/text()')[0]).strip()
                if key in dict_count:
                    dict_count[key] += 1
                else:
                    dict_count[key] = 1
            
            dict_technical, dict_no = {}, {}
            for i, tr in enumerate(list_tr):
                key = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./th/text()')[0]).strip()
                if dict_count[key] == 1:
                    dict_technical[str(i)] = {key: re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./td/text()')[0]).strip()}
                else:
                    if key in dict_no:
                        dict_no[key] += 1
                    else:
                        dict_no[key] = 1
                    dict_technical[str(i)] = {f'{key}-{dict_no[key]}': re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./td/text()')[0]).strip()}

            # = = = = = = = = = = = = = = =

            i, dict_additional, list_tr = 0, {}, html.xpath('//table[@id="productDetails_detailBullets_sections1"]/tbody/tr')
            for tr in list_tr:
                key = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./th/text()')[0]).strip()
                if key == 'Customer Reviews':
                    dict_additional[str(i)] = {'Rating': tr.xpath('./td/div/span[@data-action="acrStarsLink-click-metrics"]/span/@title')[0].strip()}
                    dict_additional[str(i+1)] = {'Rating Count': tr.xpath('./td/div/span[@data-action="acrLink-click-metrics"]/a/span/text()')[0].strip()}
                    i += 2
                elif key == 'Best Sellers Rank':
                    dict_additional[str(i)] = {key: {}}
                    list_span = [etree.tostring(span).decode('utf-8') for span in tr.xpath('./td/span/span')]
                    list_span = [re.sub(r'<.*?>|\(.*?\)', '', span).strip() for span in list_span]
                    list_href = [f'https://www.amazon.com{href.strip()}' for href in tr.xpath('./td/span/span/a/@href')]
                    for (j, span), href in zip(enumerate(list_span), list_href):
                        dict_additional[str(i)][key][str(j)] = {span: href}
                    i += 1
                else:
                    dict_additional[str(i)] = {key: re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./td/text()')[0]).strip()}
                    i += 1
        
            # = = = = = = = = = = = = = = =

            list_tr, dict_count = html.xpath('//div[@id="detailBulletsWrapper_feature_div"]/div[@id="detailBullets_feature_div"]/ul/li/span'), {}
            for tr in list_tr:
                key = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./span/text()')[0]).strip().removesuffix(':').strip()
                if key in dict_count:
                    dict_count[key] += 1
                else:
                    dict_count[key] = 1
            
            dict_technical_2, dict_no = {}, {}
            for i, tr in enumerate(list_tr):
                key = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./span/text()')[0]).strip().removesuffix(':').strip()
                if dict_count[key] == 1:
                    dict_technical_2[str(i)] = {key: re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./span/text()')[1]).strip()}
                else:
                    if key in dict_no:
                        dict_no[key] += 1
                    else:
                        dict_no[key] = 1
                    dict_technical_2[str(i)] = {f'{key}-{dict_no[key]}': re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./span/text()')[1]).strip()}

            # = = = = = = = = = = = = = = =

            i, dict_additional_2, list_span = 0, {}, html.xpath('//div[@id="detailBulletsWrapper_feature_div"]/ul[@class="a-unordered-list a-nostyle a-vertical a-spacing-none detail-bullet-list"]/li/span')
            for span in list_span:
                key = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', span.xpath('./span/text()')[0]).strip().removesuffix(':').strip()
                if key == 'Customer Reviews':
                    dict_additional_2[str(i)] = {'Rating': span.xpath('./div/span[@data-action="acrStarsLink-click-metrics"]/span/@title')[0].strip()}
                    dict_additional_2[str(i+1)] = {'Rating Count': span.xpath('./div/span[@data-action="acrLink-click-metrics"]/a/span/text()')[0].strip()}
                    i += 2
                elif key == 'Best Sellers Rank':
                    dict_additional_2[str(i)] = {key: {}}
                    list_span_2 = [etree.tostring(span_2).decode('utf-8') for span_2 in span.xpath('./descendant-or-self::span[@class="a-list-item"]')]
                    list_span_2 = [re.sub(r'<ul.*?</ul>|<span.*?Best Sellers Rank.*?</span>|<.*?>|\(.*?\)', '', span_2).strip() for span_2 in list_span_2]
                    list_href = [f'https://www.amazon.com{href.strip()}' for href in span.xpath('./descendant-or-self::a/@href')]
                    for (j, span_2), href in zip(enumerate(list_span_2), list_href):
                        dict_additional_2[str(i)][key][str(j)] = {span_2: href}
                    i += 1
                else:
                    raise

            # = = = = = = = = = = = = = = =

            list_tr, dict_count = html.xpath('//div[@id="productDetails_expanderSectionTables"]/div[@class="a-row"]/div[contains(@class, "a-column a-span6")]/div[@class="a-row a-spacing-medium-plus"]/div[@class="a-section"]/div[contains(@data-csa-c-content-id, "voyager-expander-btn")]/div/div[@class="a-section"]/table/tbody/tr'), {}
            for tr in list_tr:
                key = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./th/text()')[0]).strip().removesuffix(':').strip()
                if key in dict_count:
                    dict_count[key] += 1
                else:
                    dict_count[key] = 1
            
            i, dict_information, dict_no = 0, {}, {}
            for tr in list_tr:
                key = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./th/text()')[0]).strip().removesuffix(':').strip()
                if dict_count[key] == 1:
                    if key == 'Customer Reviews':
                        dict_information[str(i)] = {'Rating': tr.xpath('./td/div/span[@data-action="acrStarsLink-click-metrics"]/span/@title')[0].strip()}
                        dict_information[str(i+1)] = {'Rating Count': tr.xpath('./td/div/span[@data-action="acrLink-click-metrics"]/a/span/text()')[0].strip()}
                        i += 2
                    elif key == 'Best Sellers Rank':
                        dict_information[str(i)] = {key: {}}
                        list_span = [etree.tostring(span).decode('utf-8') for span in tr.xpath('./td/span/span')]
                        list_span = [re.sub(r'<.*?>|\(.*?\)', '', span).strip() for span in list_span]
                        list_href = [f'https://www.amazon.com{href.strip()}' for href in tr.xpath('./td/span/span/a/@href')]
                        for (j, span), href in zip(enumerate(list_span), list_href):
                            dict_information[str(i)][key][str(j)] = {span: href}
                        i += 1
                    else:
                        dict_information[str(i)] = {key: re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./td/text()')[0]).strip()}
                        i += 1
                else:
                    if key in dict_no:
                        dict_no[key] += 1
                    else:
                        dict_no[key] = 1
                    dict_information[str(i)] = {f'{key}-{dict_no[key]}': re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./td/text()')[0]).strip()}
                    i += 1
    
            # = = = = = = = = = = = = = = =
    
            df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                     'Title': title,
                                     'About': about.strip(),
                                     'Price': price,
                                     'Picture': '',
                                     'Url': input_.loc[a, 'Url'],
                                     'Json_Src': json.dumps(dict_src),
                                     'Zip Code': zip_code,
                                     'Parent ASIN': parent_asin,
                                     'Json_Overview': json.dumps(dict_overview),
                                     'Json_Technical': json.dumps(dict_technical),
                                     'Json_Additional': json.dumps(dict_additional),
                                     'Json_Technical 2': json.dumps(dict_technical_2),
                                     'Json_Additional 2': json.dumps(dict_additional_2),
                                     'Json_Information': json.dumps(dict_information)}])
            
            output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
            df_temp.loc[0, 'Request_Url'] = request_url

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and (len(input_) - a - 1) == 0 or (len(input_) - a - 1) % 1_000 == 0 and (len(input_) - a - 1) != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'No']}.{request_url}\n[剩余数量：{len(input_) - a - 1}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

        
    # = = = = = = = = = = = = = = =

    browser.quit()

    # = = = = = = = = = = = = = = =

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['No'],
                                                    ascending=[True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./file/2.part/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
        output_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-part_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/2.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/2.part/{file}',
              f'''./file/2.part/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：2



fuck-crawler_2-part_error?:  fuck


[状态：ok，尝试次数：1] - 100.00% - crawler_2-part_error > 3.https://www.amazon.com/dp/B0CHYBKQPM?language=en_US
[剩余数量：1] - [当前时间：13:48:44]

[状态：ok，尝试次数：1] - 100.00% - crawler_2-part_error > 69.https://www.amazon.com/dp/B0CGX5KFSF?language=en_US
[剩余数量：0] - [当前时间：13:48:49]

输出ing...



Progress: 100%|██████████████████████████████| 1/1 [00:00<00:00, 1844.46it/s]

Done ~
